In [4]:
import shutil
import subprocess
import glob
from tqdm import tqdm
import numpy as np
import cv2
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model 
#from app.fire import fire
#from elapsedtimer import ElapsedTimer

Using TensorFlow backend.



<br>
VideoCaptioningPreProcessing will extract the Dense features from the <br>
images in the Video frame using a Pre-trained VGG16 CNN Model<br>


In [5]:
class VideoCaptioningPreProcessing:
    
    def __init__(self,video_dest,feat_dir,
                 temp_dest,img_dim=224,channels=3,
                 batch_size=128,frames_step=80):
        
        self.img_dim = img_dim
        self.channels = channels
        self.video_dest = video_dest
        self.feat_dir = feat_dir
        self.temp_dest = temp_dest
        self.batch_cnn = batch_size
        self.frames_step = frames_step
        
        
    def video_to_frames(self,video):
        os.chdir('C:/Project/YouTubeClips')
        with open(os.devnull, "w") as ffmpeg_log:
            if os.path.exists(self.temp_dest):
                print(" cleanup: " + self.temp_dest + "/")
                print(video)
                shutil.rmtree(self.temp_dest)
            os.makedirs(self.temp_dest)
            video_to_frames_cmd = ["ffmpeg",                                       
                                       '-y',
                                       '-i', video,  
                                       '-vf', "scale=400:300", 
                                       '-qscale:v', "2", 
                                       '{0}/%06d.jpg'.format(self.temp_dest)]
            subprocess.call(video_to_frames_cmd,
                            stdout=ffmpeg_log, stderr=ffmpeg_log, shell=True)
                        
# Load the pre-trained VGG16 Model and extract the dense features as output 
    def model_cnn_load():
     model = VGG16(weights = "imagenet", include_top=True,input_shape = (img_dim,img_dim,channels))
     out = model.layers[-2].output
     model_final = Model(inputs=model.input,outputs=out)
     return model_final  
    
    def load_image(self,path):
        #os.chdir('C:/Project/TempImages2')
        img = cv2.imread(path)
        img = cv2.resize(img,(self.img_dim,self.img_dim))
        return img 

# Extract the features from the pre-trained CNN 
    def extract_feats_pretrained_cnn(self):

        model = model_cnn_load()
        print('Model loaded')

        if not os.path.isdir(self.feat_dir):
            os.mkdir(self.feat_dir)
        #print("save video feats to %s" % (self.dir_feat))
        video_list = glob.glob(os.path.join(self.video_dest, '*.avi'))
       # print (video_list )

        for video in tqdm(video_list):

            video_id = video.split("/")[-1].split(".")[0]
            print(f'Processing video {video}')
           #print(f'Processing video {video_id+'.avi'}

            #self.dest = 'cnn_feat' + '_' + video_id
            self.video_to_frames(video)
          
            image_list = sorted(glob.glob(os.path.join(self.temp_dest, '*.jpg')))
            samples = np.round(np.linspace( 0, len(image_list) - 1,self.frames_step))
            image_list =[image_list[int(sample)] for sample in samples]
            images = np.zeros((len(image_list),self.img_dim,self.img_dim,
                     self.channels))
            for i in range(len(image_list)):
                img = self.load_image(image_list[i])
                images[i] = img
            images = np.array(images)
            print('Model predicting')
            fc_feats = model.predict(images,batch_size=self.batch_cnn)
            print('prediction finished')
            img_feats = np.array(fc_feats)
            outfile = os.path.join(self.feat_dir, video_id + '.npy')
            print('features saved')
            np.save(outfile, img_feats)
            # cleanup
            shutil.rmtree(self.temp_dest)
        

In [6]:
video_dest = r'C:\Project\YouTubeClips'
feat_dir =  'C:\\Project\\features2'
temp_dest = 'C:\\Project\\TempImages2\\'
img_dim = 224
channels=3
batch_size=128
batch_cnn =128
frames_step=80


In [7]:
def model_cnn_load():
     model = VGG16(weights = "imagenet", include_top=True,input_shape = (img_dim,img_dim,channels))
     out = model.layers[-2].output
     model_final = Model(inputs=model.input,outputs=out)
     return model_final  
#model = model_cnn_load()

Convert the video into image frames at a specified sampling rate 

In [ ]:
 extract_features = VideoCaptioningPreProcessing(video_dest=video_dest,feat_dir=feat_dir,temp_dest=temp_dest,img_dim=img_dim,channels=channels, batch_size=128, frames_step=80)
#extract_features.model_cnn_load()
extract_features.extract_feats_pretrained_cnn()

Instructions for updating:
Colocations handled automatically by placer.
Model loaded


  0%|          | 0/1970 [00:00<?, ?it/s]

Processing video C:\Project\YouTubeClips\-4wsuPCjDBc_5_15.avi
 cleanup: C:\Project\TempImages2\/
C:\Project\YouTubeClips\-4wsuPCjDBc_5_15.avi
Model predicting


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sys
import time
import cv2
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn
#from app.fire import  fire
#from elapsedtimer import ElapsedTimer
from pathlib import Path
print('tensorflow version:',tf.__version__)
from IPython.core.debugger import Pdb
ipdb = Pdb()

Using TensorFlow backend.


tensorflow version: 1.13.1


This takes the CNN features of a video frames and passes it through Back to Back LSTMs(Sequence to Sequence
Model) to generate the Caption for the Video
path_prj - Project directory.
feat_dir - Subdirectory containing the CNN features .. absolute path /path_prj/feat_dir/
cnn_feat_dim - Dimension of the feature vector from CNN for each image frame 
video_steps - No of image frames from each video. 
out_steps - Sequence length for the text caption. The output text sequence would be contained in 2o words.
learning rate - training hyper parameter
epoch - Traing epochs
model_path - Absolute Path to save the model 
mode - train/inference

In [3]:
class VideoCaptioning:
    
    
    def __init__(self,path_prj,caption_file,feat_dir,
                 cnn_feat_dim=4096,h_dim=512,
                 lstm_steps=80,video_steps=80,
                 out_steps=20, frame_step=80,
                 batch_size=8,learning_rate=1e-4,
                 epochs=100,model_path=None,
                 mode='train'):
        self.dim_image = cnn_feat_dim
        self.dim_hidden = h_dim
        self.batch_size = batch_size
        self.lstm_steps = lstm_steps
        self.video_lstm_step=video_steps
        self.caption_lstm_step=out_steps
        self.path_prj = Path(path_prj)
        self.mode = mode
        if mode == 'train':
            self.train_text_path = self.path_prj / caption_file
            self.train_feat_path = self.path_prj / feat_dir
        else:
            self.test_text_path = self.path_prj / caption_file
            self.test_feat_path = self.path_prj / feat_dir
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.frame_step = frame_step
        self.model_path = model_path
    def build_model(self):

        # Defining the weights associated with the Network
        with tf.device('/cpu:0'): 
            self.word_emb = tf.Variable(tf.random_uniform([self.n_words, self.dim_hidden], -0.1, 0.1), name='word_emb')
        self.lstm1 = tf.nn.rnn_cell.BasicLSTMCell(self.dim_hidden, state_is_tuple=False)
        self.lstm2 = tf.nn.rnn_cell.BasicLSTMCell(self.dim_hidden, state_is_tuple=False)
        self.encode_W = tf.Variable( tf.random_uniform([self.dim_image,self.dim_hidden], -0.1, 0.1), name='encode_W')
        self.encode_b = tf.Variable( tf.zeros([self.dim_hidden]), name='encode_b')
        
        self.word_emb_W = tf.Variable(tf.random_uniform([self.dim_hidden,self.n_words], -0.1,0.1), name='word_emb_W')
        self.word_emb_b = tf.Variable(tf.zeros([self.n_words]), name='word_emb_b')
        
        # Placeholders 
        video = tf.placeholder(tf.float32, [self.batch_size, self.video_lstm_step, self.dim_image])
        video_mask = tf.placeholder(tf.float32, [self.batch_size, self.video_lstm_step])
        caption = tf.placeholder(tf.int32, [self.batch_size, self.caption_lstm_step+1])
        caption_mask = tf.placeholder(tf.float32, [self.batch_size, self.caption_lstm_step+1])
        video_flat = tf.reshape(video, [-1, self.dim_image])
        image_emb = tf.nn.xw_plus_b( video_flat, self.encode_W,self.encode_b )         
        image_emb = tf.reshape(image_emb, [self.batch_size, self.lstm_steps, self.dim_hidden])
        state1 = tf.zeros([self.batch_size, self.lstm1.state_size])
        state2 = tf.zeros([self.batch_size, self.lstm2.state_size])
        padding = tf.zeros([self.batch_size, self.dim_hidden])
        probs = []
        loss = 0.0

        #  Encoding Stage 
        for i in range(0, self.video_lstm_step):
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            with tf.variable_scope("LSTM1"):
                output1, state1 = self.lstm1(image_emb[:,i,:], state1)
            with tf.variable_scope("LSTM2"):
                output2, state2 = self.lstm2(tf.concat([padding, output1],1), state2)

        #  Decoding Stage  to generate Captions 
        for i in range(0, self.caption_lstm_step):
            with tf.device("/cpu:0"):
                current_embed = tf.nn.embedding_lookup(self.word_emb, caption[:, i])
            tf.get_variable_scope().reuse_variables()
            with tf.variable_scope("LSTM1"):
                output1, state1 = self.lstm1(padding, state1)
            with tf.variable_scope("LSTM2"):
                output2, state2 = self.lstm2(tf.concat([current_embed, output1],1), state2)
            labels = tf.expand_dims(caption[:, i+1], 1)
            indices = tf.expand_dims(tf.range(0, self.batch_size, 1), 1)
            concated = tf.concat([indices, labels],1)
            onehot_labels = tf.sparse_to_dense(concated, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)
            logit_words = tf.nn.xw_plus_b(output2, self.word_emb_W, self.word_emb_b)
        # Computing the loss     
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit_words,labels=onehot_labels)
            cross_entropy = cross_entropy * caption_mask[:,i]
            probs.append(logit_words)
            current_loss = tf.reduce_sum(cross_entropy)/self.batch_size
            loss = loss + current_loss
        with tf.variable_scope(tf.get_variable_scope(),reuse=tf.AUTO_REUSE):
            train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(loss)    
        return loss, video, video_mask, caption, caption_mask, probs,train_op

In [4]:
    def build_generator(self):
        with tf.device('/cpu:0'):
            self.word_emb = tf.Variable(tf.random_uniform([self.n_words, self.dim_hidden], -0.1, 0.1), name='word_emb')
            self.lstm1 = tf.nn.rnn_cell.BasicLSTMCell(self.dim_hidden, state_is_tuple=False)
            self.lstm2 = tf.nn.rnn_cell.BasicLSTMCell(self.dim_hidden, state_is_tuple=False)
            self.encode_W = tf.Variable( tf.random_uniform([self.dim_image,self.dim_hidden], -0.1, 0.1), name='encode_W')
            self.encode_b = tf.Variable( tf.zeros([self.dim_hidden]), name='encode_b')
            self.word_emb_W = tf.Variable(tf.random_uniform([self.dim_hidden,self.n_words], -0.1,0.1), name='word_emb_W')
            self.word_emb_b = tf.Variable(tf.zeros([self.n_words]), name='word_emb_b')
            video = tf.placeholder(tf.float32, [1, self.video_lstm_step, self.dim_image])
            video_mask = tf.placeholder(tf.float32, [1, self.video_lstm_step])
            video_flat = tf.reshape(video, [-1, self.dim_image])
            image_emb = tf.nn.xw_plus_b(video_flat, self.encode_W, self.encode_b)
            image_emb = tf.reshape(image_emb, [1, self.video_lstm_step, self.dim_hidden])
            state1 = tf.zeros([1, self.lstm1.state_size])
            state2 = tf.zeros([1, self.lstm2.state_size])
            padding = tf.zeros([1, self.dim_hidden])
            generated_words = []
            probs = []
            embeds = []
            for i in range(0, self.video_lstm_step):
                if i > 0:
                    tf.get_variable_scope().reuse_variables()
                with tf.variable_scope("LSTM1"):
                    output1, state1 = self.lstm1(image_emb[:, i, :], state1)
                with tf.variable_scope("LSTM2"):
                    output2, state2 = self.lstm2(tf.concat([padding, output1],1), state2)
            for i in range(0, self.caption_lstm_step):
                tf.get_variable_scope().reuse_variables()
                if i == 0:
                    with tf.device('/cpu:0'):
                        current_embed = tf.nn.embedding_lookup(self.word_emb, tf.ones([1], dtype=tf.int64))
                with tf.variable_scope("LSTM1"):
                    output1, state1 = self.lstm1(padding, state1)
                with tf.variable_scope("LSTM2"):
                    output2, state2 = self.lstm2(tf.concat([current_embed, output1],1), state2)
                logit_words = tf.nn.xw_plus_b( output2, self.word_emb_W, self.word_emb_b)
                max_prob_index = tf.argmax(logit_words, 1)[0]
                generated_words.append(max_prob_index)
                probs.append(logit_words)
                with tf.device("/cpu:0"):
                    current_embed = tf.nn.embedding_lookup(self.word_emb, max_prob_index)
                    current_embed = tf.expand_dims(current_embed, 0)
                embeds.append(current_embed)
            return video, video_mask, generated_words, probs, embeds

Load the video images 